In [14]:
import re
from faker import Faker
import pandas as pd
from tqdm.notebook import tqdm_notebook
from datetime import datetime

In [4]:
splitted_chat = []
with open("./chats/chats.txt", "r") as chat:
    all_chat = chat.read().replace("\n", " ")
    splitted_chat = re.findall("\d{1,2}\/\d{1,2}\/\d{4}, \d{1,2}:\d{2} - \+[()\d\w\s-]+: [\s\S]*?(?=\d{1,2}\/\d{1,2}\/\d{4}, \d{1,2}:\d{2} - \+|$)", all_chat)

In [25]:
faker = Faker()
dict_of_names = {}

In [26]:
def process_line(line):
    #10/3/2024, 12:44 - +54 9 11 3763-1285: Muchas Gracias !!! Ya compre entradas.
    regex_result = re.search("^(\d{1,2}\/\d{1,2}\/\d{4}), (\d\d:\d\d) - (\+(\d*) (?:9 (\d*)|([()\d\w\s-]*))([()\d\w\s-]*)): ([\s\S]*)$", line)
    date = regex_result.group(1)
    time = regex_result.group(2)
    phone_number = regex_result.group(3)
    message = regex_result.group(8)
    date = datetime.strptime(f"{date} {time}", "%d/%m/%Y %H:%M")
    return pd.DataFrame([{
            "phone": phone_number,
            "fake_name": dict_of_names.setdefault(phone_number, faker.name()),
            "text": message.lower(),
            "date": date,
           }])

In [27]:
after_process = pd.DataFrame()
with tqdm_notebook(total=len(splitted_chat)) as progress_bar:
    for line in splitted_chat:
        try:
            processed_line = process_line(line)
            after_process = pd.concat([after_process, processed_line])
        except Exception as err:
            print(f"EXCEPTION {err} on line {line}")
            pass
        finally:
            progress_bar.update(1)
            
after_process.to_csv("./chats/dataset.csv")

  0%|          | 0/7284 [00:00<?, ?it/s]

In [28]:
after_process

,phone,fake_name,text,date
0,+54 9 11 3763-1285,Cristian Thomas,muchas gracias !!! ya compre entradas.,2024-03-10 12:44:00
0,+54 9 11 6544-3716,Ann Perry,https://eternacadencia.com.ar/nota/borges-pres...,2024-03-10 14:07:00
0,+54 9 11 6544-3716,Ann Perry,"borges , el lector conmovido",2024-03-10 14:09:00
0,+593 98 758 1373,Rodney Gonzalez,https://almomento.net/cecilia-ingenieros-borge...,2024-03-10 17:03:00
0,+593 98 758 1373,Rodney Gonzalez,"muy interesante información sobre ""emma zunz"" ...",2024-03-10 17:04:00
...,...,...,...,...
0,+54 9 2314 41-1016,Amanda Delgado,me encantò!!! gracias sss 💚,2024-05-10 16:28:00
0,+54 9 2314 41-1016,Amanda Delgado,es maravilloso este grupo! y qué placer la lec...,2024-05-10 16:30:00
0,+54 9 11 4084-4598,Austin Jackson,https://youtu.be/dbxlrkmfm8q?si=hvwilfqkp8fb_4yx,2024-05-10 16:57:00
0,+54 9 11 4084-4598,Austin Jackson,"no sé si alguien lo compartió ya, pero es un v...",2024-05-10 16:58:00
